<a href="https://colab.research.google.com/github/eliot28/desafioLatam/blob/main/Tutoria_4_ML_RegreLog_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [3]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

df = pd.concat([test,train])
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,NaN
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,NaN
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,NaN
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,NaN
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
 11  Survived     891 non-null    float64
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [5]:
df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Survived
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,891.000000
mean,655.000000,2.294882,29.881138,0.498854,0.385027,33.295479,0.383838
std,378.020061,0.837836,14.413493,1.041658,0.865560,51.758668,0.486592
min,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,328.000000,2.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,655.000000,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,982.000000,3.000000,39.000000,1.000000,0.000000,31.275000,1.000000
max,1309.000000,3.000000,80.000000,8.000000,9.000000,512.329200,1.000000


In [7]:
df.shape

(1309, 12)

In [8]:
# tipo, valores unicos, valores faltantes
df.dtypes

,0
PassengerId,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64
Cabin,object


In [9]:
###

categorical_features = df.select_dtypes(include=[object]).columns.tolist()
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
print(categorical_features)
print(numeric_features)


['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']


In [23]:
df_final = pd.get_dummies(df.loc[:,['Sex', 'Embarked']]).astype('int64')
df_final = df_final.drop(['Sex_female'],axis=1)
df_final.head()

,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,1,0
1,0,0,0,1
2,1,0,1,0
3,1,0,0,1
4,0,0,0,1


In [24]:
df_final = pd.concat([df_final,df[numeric_features]],axis=1)
df_final.head()

,Sex_male,Embarked_C,Embarked_Q,Embarked_S,PassengerId,Pclass,Age,SibSp,Parch,Fare,Survived
0,1,0,1,0,892,3,34.5,0,0,7.8292,NaN
1,0,0,0,1,893,3,47.0,1,0,7.0000,NaN
2,1,0,1,0,894,2,62.0,0,0,9.6875,NaN
3,1,0,0,1,895,3,27.0,0,0,8.6625,NaN
4,0,0,0,1,896,3,22.0,1,1,12.2875,NaN


In [25]:
df_final = df_final.drop(['SibSp','SibSp'],axis=1)

In [28]:
df_final['Survived'].value_counts(dropna=False)

,count
Survived,
0.0,549
NaN,418
1.0,342


In [31]:
df_final = df_final.dropna()
df_final['Survived'].value_counts(dropna=False)

,count
Survived,
0.0,424
1.0,290


In [32]:
X = df_final.drop('Survived',axis=1)
y = df_final['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [33]:
# Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [34]:
X_train.head()

,Sex_male,Embarked_C,Embarked_Q,Embarked_S,PassengerId,Pclass,Age,Parch,Fare
37,1,0,0,1,38,3,21.0,0,8.0500
58,0,0,0,1,59,2,5.0,2,27.7500
57,1,1,0,0,58,3,28.5,0,7.2292
94,1,0,0,1,95,3,59.0,0,7.2500
209,1,1,0,0,210,1,40.0,0,31.0000


In [36]:
X_train_scaled

array([[ 0.74791377, -0.46591112, -0.20960213, ..., -0.62918643,
        -0.49495371, -0.58475221],
       [-1.33705254, -0.46591112, -0.20960213, ..., -1.70607682,
         1.77093529, -0.15874221],
       [ 0.74791377,  2.14633212, -0.20960213, ..., -0.12439406,
        -0.49495371, -0.60250191],
       ...,
       [-1.33705254,  2.14633212, -0.20960213, ...,  0.71692656,
        -0.49495371,  2.14971302],
       [ 0.74791377, -0.46591112, -0.20960213, ...,  0.17848136,
         0.63799079, -0.31498192],
       [ 0.74791377,  2.14633212, -0.20960213, ...,  0.17848136,
        -0.49495371, -0.58808677]])

In [38]:
logistic_model = LogisticRegression()

param_grid_logistic = {'C': [0.01, 0.1, 1, 10, 100.]}

grid_search_lm = GridSearchCV(logistic_model,param_grid_logistic, cv=5)
grid_search_lm.fit(X_train_scaled, y_train)
best_l_model = grid_search_lm.best_estimator_

grid_search_lm.best_params_

{'C': 1}

In [40]:
y_pred_logistic = best_l_model.predict(X_test_scaled)
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
precision_logistic = precision_score(y_test, y_pred_logistic)
recall_logistic = recall_score(y_test, y_pred_logistic)
f1_score_logistic = f1_score(y_test, y_pred_logistic)
roc_auc_logistic = roc_auc_score(y_test, best_l_model.predict_proba(X_test_scaled)[:, 1])

print("Regresión Logística:")
print("Accuracy:", accuracy_logistic)
print("Precision:", precision_logistic)
print("Recall:", recall_logistic)
print("F1 Score:", f1_score_logistic)
print("AUC-ROC:", roc_auc_logistic)

Regresión Logística:
Accuracy: 0.7441860465116279
Precision: 0.6976744186046512
Recall: 0.6741573033707865
F1 Score: 0.6857142857142857
AUC-ROC: 0.8214731585518102


In [ ]:
svm_model = SVC(probability=True)

# Búsqueda de hiperparámetros para SVM
param_grid_svm = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10], 'kernel':['linear', 'rbf', 'poly']}
grid_search_svm = GridSearchCV(svm_model, param_grid_svm, cv=5)
grid_search_svm.fit(X_train_scaled, y_train)
best_svm_model = grid_search_svm.best_estimator_

grid_search_svm.best_params_